# =============================================================================
# Сборщик научно-популярного русскоязычного корпуса текстов (2025–2026)
# Темы: ИИ, нейросети, космос, биотехнологии, квантовая физика, климат, энергетика
# =============================================================================

In [ ]:


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# ========================================
# Установка зависимостей (если нужно)
# ========================================
# %pip install -q requests fake-useragent beautifulsoup4 pymorphy3 nltk pandas tqdm

import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time
import random
import re
import json
from datetime import datetime
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy3
from urllib.parse import quote_plus, urlparse
import pandas as pd
from tqdm import tqdm

print("Инициализация...")

# NLTK ресурсы
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

# Морфология
morph = pymorphy3.MorphAnalyzer()
ru_stop = set(stopwords.words('russian'))

print("Библиотеки готовы\n")

# ========================================
# СПИСОК ТЕМ И КЛЮЧЕВЫХ СЛОВ (2025–2026)
# ========================================
THEMES = [
    "искусственный интеллект 2025", "нейросети 2026", "генеративный ИИ",
    "квантовые компьютеры", "квантовая криптография", "квантовые вычисления",
    "термоядерный синтез", "ядерный синтез 2026", "энергия будущего",
    "изменение климата 2025", "углеродная нейтральность", "зелёная энергетика",
    "биотехнологии", "генетическое редактирование", "CRISPR 2026",
    "космические миссии 2025", "колонизация Марса", "Starship", "лунная база",
    "нейроморфные чипы", "brain-computer interface", "нейроинтерфейсы",
    "метавселенная 2026", "Web3", "децентрализованные финансы",
    "робототехника человекоподобные", "гуманоидные роботы",
    "автономные автомобили 2026", "беспилотники уровень 5",
    "долгожительство", "биохакинг", "омоложение",
    "космический туризм", "орбитальные отели", "SpaceX",
    "водородная энергетика", "батареи нового поколения", "твердотельные аккумуляторы"
]

# ========================================
# ПОИСК ЧЕРЕЗ ЯНДЕКС (основной источник)
# ========================================
def search_yandex(query, count=20):
    try:
        url = f"https://yandex.ru/search/?text={quote_plus(query)}&lr=213"
        headers = {'User-Agent': UserAgent().random}
        r = requests.get(url, headers=headers, timeout=12)
        soup = BeautifulSoup(r.text, 'html.parser')

        links = []
        for item in soup.select('li.serp-item a.Link'):
            href = item.get('href')
            if href and href.startswith('http') and 'yandex' not in href:
                links.append(href)

        print(f"Яндекс → {len(links)} ссылок по '{query}'")
        return links[:count]
    except Exception as e:
        print(f"Ошибка Яндекса: {e}")
        return []

# ========================================
# БАЗОВЫЕ НАУЧНЫЕ САЙТЫ (прямые ссылки)
# ========================================
def get_seed_urls():
    seeds = []
    sites = [
        "https://nplus1.ru/search?query={q}",
        "https://habr.com/ru/search/?q={q}",
        "https://naked-science.ru/search?query={q}",
        "https://elementy.ru/search?query={q}",
        "https://postnauka.ru/search?query={q}",
        "https://scientificrussia.ru/search?query={q}",
        "https://trends.rbc.ru/search?query={q}",
        "https://vc.ru/search/v2/content?query={q}",
    ]

    for q in THEMES[:30]:
        for site in sites:
            try:
                url = site.format(q=quote_plus(q))
                seeds.append(url)
            except:
                pass
    return seeds

# ========================================
# СОБИРАЕМ URL
# ========================================
def collect_links(target=1200):
    print(f"Собираем ссылки... Цель: {target}")
    urls = set()

    # 1. Прямые запросы по темам
    for theme in tqdm(THEMES, desc="По темам"):
        links = search_yandex(theme, 25)
        urls.update(links)
        time.sleep(random.uniform(1.8, 4.2))

    # 2. Семенные сайты
    print("\nДобавляем прямые ссылки с научных сайтов...")
    seeds = get_seed_urls()
    urls.update(seeds)

    # Фильтрация мусора
    bad = {'youtube.com', 'vk.com', 't.me', 'facebook.com', 'instagram.com', 'pikabu.ru'}
    filtered = [u for u in urls if not any(x in u for x in bad)]

    print(f"Всего собрано уникальных: {len(filtered)}")
    return filtered[:target]

# ========================================
# ПАРСИНГ СТРАНИЦЫ
# ========================================
def scrape_page(url, session):
    try:
        r = session.get(url, timeout=14)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, 'lxml')

        # Удаляем мусор
        for el in soup(['script', 'style', 'nav', 'footer', 'header', 'aside', 'form', 'iframe']):
            el.decompose()

        title = (soup.title.string or "Без заголовка").strip()

        # Текст — самые вероятные контейнеры
        candidates = soup.find_all(['article', 'div', 'section'], class_=re.compile(r'(article|post|content|entry|text|body|main)'))
        if not candidates:
            candidates = soup.find_all('p')

        paragraphs = [p.get_text(strip=True) for p in candidates if len(p.get_text(strip=True)) > 40]
        text = ' '.join(paragraphs)

        if len(text) < 300:
            return None

        # Дата (попытка)
        date_tag = soup.find('meta', {'property': 'article:published_time'}) or \
                   soup.find('meta', {'name': 'date'}) or \
                   soup.find('time', {'datetime': True})

        date = datetime.now().strftime('%Y-%m-%d')
        if date_tag:
            dt = date_tag.get('content') or date_tag.get('datetime')
            if dt:
                try:
                    date = datetime.fromisoformat(dt.split('T')[0]).strftime('%Y-%m-%d')
                except:
                    pass

        return {'url': url, 'title': title, 'text': text[:12000], 'date': date}
    except:
        return None

# ========================================
# ОЧИСТКА И ЛЕММАТИЗАЦИЯ
# ========================================
def process_text(raw):
    text = re.sub(r'<[^>]+>', ' ', raw)
    text = re.sub(r'\d+[.,]?\d*', ' ', text)
    text = re.sub(r'[«»—–…“”„‟′‶`´]+', ' ', text)
    text = re.sub(r'\s+', ' ', text.strip())

    tokens = word_tokenize(text.lower())
    lemmas = []
    for t in tokens:
        if t.isalpha() and len(t) > 2 and t not in ru_stop:
            p = morph.parse(t)[0]
            lemmas.append(p.normal_form)

    return ' '.join(lemmas), len(lemmas)

# ========================================
# ОСНОВНОЙ ЦИКЛ
# ========================================
def run_scraper(max_docs=800):
    print(f"\n=== СБОР НАУЧНО-ПОП КОРПУСА 2025–2026 ===\nЦель: {max_docs} документов\n")

    session = requests.Session()
    session.headers['User-Agent'] = UserAgent().random

    links = collect_links(target=max_docs * 2)
    random.shuffle(links)

    documents = []
    seen = set()

    for url in tqdm(links, desc="Парсинг страниц"):
        if len(documents) >= max_docs:
            break
        if url in seen:
            continue

        data = scrape_page(url, session)
        if not data:
            continue

        cleaned, count = process_text(data['text'])
        if count < 40:
            continue

        documents.append({
            'id': len(documents) + 1,
            'title': data['title'][:220],
            'url': url,
            'date': data['date'],
            'raw_length': len(data['text']),
            'tokens': count,
            'cleaned': cleaned,
            'source': urlparse(url).netloc
        })

        seen.add(url)
        time.sleep(random.uniform(1.1, 3.3))

    print(f"\nСобрано документов: {len(documents)}")

    # Сохранение
    df = pd.DataFrame(documents)
    df.to_csv('science_corpus_2026.csv', index=False, encoding='utf-8-sig')
    print("→ CSV сохранён")

    with open('science_corpus_2026.json', 'w', encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=2)
    print("→ JSON сохранён")

    # TXT-версия для чтения
    with open('science_corpus_2026.txt', 'w', encoding='utf-8') as f:
        for doc in documents:
            f.write(f"{'='*90}\n")
            f.write(f"ID {doc['id']} | {doc['date']} | {doc['source']}\n")
            f.write(f"{doc['title']}\n")
            f.write(f"URL: {doc['url']}\n")
            f.write(f"Токенов: {doc['tokens']}\n")
            f.write(f"{'-'*90}\n")
            f.write(f"{doc['cleaned'][:1500]}...\n\n")

    print("→ TXT сохранён")

    # Статистика
    print("\n" + "="*60)
    print("СТАТИСТИКА КОРПУСА")
    print("="*60)
    print(f"Документов:          {len(documents)}")
    print(f"Общее токенов:       {df['tokens'].sum():,}")
    print(f"Среднее токенов/док: {df['tokens'].mean():.1f}")
    print(f"Уникальных источников: {df['source'].nunique()}")
    print("\nТоп-10 источников:")
    print(df['source'].value_counts().head(10))

    print("\nГотово! Корпус можно использовать для тематического моделирования, классификации и т.д.")

if __name__ == '__main__':
    run_scraper(max_docs=750)